In [11]:
from tqdm import tqdm
import gzip
from collections import namedtuple
from glob import glob


In [92]:
split_tsv = "output/results/N1_dev/freddie.split/21.split.tsv.gz"
old_splits = "output/results/N1/freddie.split/21/split_*.tsv.gz"

In [117]:
Tint = namedtuple(
    "Tint",
    [
        "idx",
        "contig",
        "reads",
        "intervals",
    ],
)
Read = namedtuple(
    "Read",
    [
        "idx",
        "name",
        "intervals",
    ],
)

In [158]:
new_tints = list()
for line in tqdm(gzip.open(split_tsv, "rt")):
    line = line.rstrip("\n")
    if line.startswith("#"):
        line = line.lstrip("#")
        contig, tint_id, read_count, intervals = line.split("\t")
        intervals = intervals.split(",")
        intervals = tuple(
            tuple(map(int, interval.split("-"))) for interval in intervals
        )
        new_tints.append(
            Tint(
                idx=int(tint_id),
                contig=contig,
                reads=dict(),
                intervals=intervals,
            )
        )
        continue
    line = line.split("\t")
    ridx, name, strand = line[:3]
    ridx = int(ridx)
    intervals = line[3:]
    intervals = tuple(
        tuple(
            int(pos)
            for interval in paired_interval.split(":")
            for pos in interval.split("-")
        )
        for paired_interval in intervals[2:]
    )
    new_tints[-1].reads[name] = Read(
        idx=ridx,
        name=name,
        intervals=intervals,
    )

87994it [00:01, 77370.92it/s] 


In [159]:
old_tints = list()
for path in glob(old_splits):
    f = gzip.open(path, "rt")
    line = f.readline()
    line = line.rstrip("\n")
    line = line.lstrip("#")
    contig, tint_id, intervals, read_count = line.split("\t")
    tint_id = int(tint_id)
    intervals = intervals.split(",")
    intervals = tuple(tuple(map(int, interval.split("-"))) for interval in intervals)
    read_count = int(read_count)
    old_tints.append(
        Tint(
            idx=tint_id,
            contig=contig,
            reads=dict(),
            intervals=intervals,
        )
    )
    for line in f:
        line = line.rstrip("\n")
        line = line.split("\t")
        ridx, name, contig, strand, tint_id = line[:5]
        intervals = line[5:]
        ridx = int(ridx)
        intervals = tuple(
            tuple(
                int(pos)
                for interval in paired_interval.split(":")[:2]
                for pos in interval.split("-")
            )
            for paired_interval in intervals
        )
        old_tints[-1].reads[name] = Read(
            idx=ridx,
            name=name,
            intervals=intervals,
        )
    assert len(old_tints[-1].reads) == read_count, (
        len(old_tints[-1].reads),
        read_count,
    )

In [149]:
from collections import defaultdict


name_to_intervals = defaultdict(lambda: [tuple(), tuple()])
for tint in old_tints:
    for read in tint.reads.values():
        name_to_intervals[read.name][0] = read.intervals
for tint in new_tints:
    for read in tint.reads.values():
        name_to_intervals[read.name][1] = read.intervals

In [214]:
mapping = defaultdict()
for ot in tqdm(old_tints):
    mapping[ot.idx] = list()
    if len([r for r in ot.reads.values() if len(r.intervals)>1]) <= 3:
        continue
    if len(ot.intervals) == 1:
        continue
    for nt in new_tints:
        if set(ot.reads) & set(nt.reads) == set():
            continue
        mapping[ot.idx].append(nt.idx)
    assert len(mapping[ot.idx]) == 1, (ot.idx, mapping[ot.idx])

100%|██████████| 279/279 [00:07<00:00, 39.39it/s]


In [38]:
import re

interval_re = "([0-9]+)-([0-9]+)"
rinterval_re = "%(i)s:%(i)s" % {"i": interval_re}
chr_re = "[0-9A-Za-z!#$%&+./:;?@^_|~-][0-9A-Za-z!#$%&*+./:;=?@^_|~-]*"
tint_interval_prog = re.compile(interval_re)
read_interval_prog = re.compile(rinterval_re)
tint_prog = re.compile(
    r"#%(contig)s\t" % {"contig": f"(?P<contig>{chr_re})"}
    + r"%(idx)s\t" % {"idx": "(?P<tint_id>[0-9]+)"}
    + r"%(count)s\t" % {"count": "(?P<read_count>[0-9]+)"}
    + r"%(intervals)s\n"
    % {"intervals": "(?P<intervals>%(i)s(,%(i)s)*)" % {"i": interval_re}}
)
read_prog = re.compile(
    r"%(idx)s\t" % {"idx": "(?P<rid>[0-9]+)"}
    + r"%(name)s\t" % {"name": "(?P<name>[!-?A-~]{1,254})"}
    + r"%(strand)s\t" % {"strand": "(?P<strand>[+-])"}
    + r"%(intervals)s\n$"
    % {"intervals": r"(?P<intervals>%(i)s(\t%(i)s)*)" % {"i": rinterval_re}}
)

f = gzip.open("output/results/N1_dev/freddie.split/21.split.tsv.gz", "rt")
t_line = f.readline()
r_line = f.readline()

r_line = '0\tfbae1395-b4b8-4078-a7fe-7557c1d38b1d\t+\t0-2:18-20\t0-0:0-0\t5022530-5022693:54-206\t5026253-5026363:206-327\n'

read_prog.match(r_line).groupdict()

{'rid': '0',
 'name': 'fbae1395-b4b8-4078-a7fe-7557c1d38b1d',
 'strand': '+',
 'intervals': '0-2:18-20\t0-0:0-0\t5022530-5022693:54-206\t5026253-5026363:206-327'}

In [22]:
tint_prog

re.compile(r'#(?P<contig>[0-9A-Za-z!#$%&+./:;?@^_|~-][0-9A-Za-z!#$%&*+./:;=?@^_|~-]*)\t(?P<tint_id>[0-9]+)\t(?P<read_count>[0-9]+)\t$(?P<intervals>([0-9]+)-([0-9]+)(,([0-9]+)-([0-9]+))*)\n',
           re.UNICODE)

In [213]:
from cgranges import cgranges

CR = cgranges()
S = float("inf")
E = float("-inf")
for tint in old_tints:
    if tint.idx not in [10, 9]:
        continue
    # print(tint.intervals)
    for s, e in tint.intervals:
        CR.add("", s, e, tint.idx)
        S = min(S, s)
        E = max(E, e)
    for read in tint.reads.values():
        for s, e, _, _ in read.intervals:
            CR.add("", s, e, tint.idx)
            S = min(S, s)
            E = max(E, e)
CR.index()
for x in CR.overlap("", S, E):
    print(x)

(8229019, 8229364, 9)
(8229019, 8229364, 9)
(8229961, 8229993, 10)
(8229961, 8229993, 10)
(8230270, 8230476, 9)
(8230270, 8230691, 9)
(8230307, 8230691, 9)
(8232652, 8232950, 9)
(8232652, 8232779, 9)
(8232672, 8232950, 9)
(8232736, 8232883, 9)
(8233331, 8233794, 9)
(8233331, 8233794, 9)
(8233382, 8233433, 9)
(8233795, 8233929, 9)
(8233795, 8233929, 9)
(8234035, 8234258, 10)
(8234035, 8234258, 10)
(8234197, 8234205, 10)
(8234350, 8234840, 10)
(8234350, 8234761, 10)
(8234637, 8234840, 10)
(8234866, 8235178, 9)
(8234866, 8235178, 9)
(8237716, 8237752, 10)
(8237716, 8237752, 10)
(8403027, 8403207, 9)
(8403027, 8403207, 9)
(8403795, 8403939, 9)
(8403795, 8403939, 9)
(8411679, 8411764, 9)
(8411679, 8411764, 9)
(8411941, 8412138, 9)
(8411941, 8412138, 9)
(8412288, 8412425, 9)
(8412288, 8412425, 9)
(8412499, 8412590, 9)
(8412499, 8412590, 9)
(8412760, 8413044, 9)
(8412760, 8412938, 9)
(8412829, 8413044, 9)
(8413240, 8413293, 10)
(8413240, 8413293, 10)
(8415212, 8415611, 9)
(8415212, 8415470, 9